In [2]:
import sys
import os

sys.path.append(os.path.abspath("src"))  # Ensure Python knows about 'src/'

In [3]:
from src.train import Trainer

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-03-31 10:40:49.437595: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 10:40:49.462109: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-31 10:40:49.854477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find Tensor

In [ ]:
# Set dataset directory
data_dir = "/cluster/projects/vc/data/mic/open/HNTS-MRG"

# Initialize Trainer
trainer = Trainer(
    data_dir=data_dir,
    batch_size=3,
    learning_rate=2e-3,
    early_stop_count=5,
    epochs=10, 
    validation_interval=10,
    in_channels=1, 
    out_channels=1
)

# Train the model
trainer.train()

In [ ]:
# Save the final model
trainer.save_model("results/unet3d_mri.pth")